## **Beginning**

In [ ]:
import requests
f = open("title.txt", "r")
scripts = f.read().splitlines()
f.close()
wiki = []
for script in scripts:
    print(script + " Film")
    url = "https://en.wikipedia.org/w/api.php?action=query&origin=*&format=json&generator=search&gsrnamespace=0&gsrlimit=5&gsrsearch='{}'".format(script + " Film")
    response = requests.get(url)
    data = response.json()
    pages = data['query']['pages']
    for page in pages.keys():
        if pages[page]['index'] == 1:
            print(pages[page]['pageid'])
            newurl = "https://en.wikipedia.org/w/api.php?action=query&prop=info&pageids={}&inprop=url&format=json".format(pages[page]['pageid'])
            newresponse = requests.get(newurl)
            newdata = newresponse.json()
            wiki.append(newdata["query"]["pages"][page]["fullurl"])
print(wiki)

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

script_data = pd.DataFrame(columns=['Title', 'Plot', 'Cast', 'Director', 'Writer', 'Production', 'Runtime', 'Budget', 'Box office', 'Rating', 'Genre'])

def get_movie_details(index):
    try:
        script_data.loc[index] = [np.nan] * 11
        print(scripts[index])
        print(wiki[index])
        script_data['Title'][index] = scripts[index]
        response = requests.get(wiki[index])
        soup = BeautifulSoup(response.text, 'lxml')

        for section in soup.find_all('h2'):
            if("Plot" in section.get_text()):
                plot = ""
                for sibling in section.find_next_siblings():
                    if sibling.name == 'h2':
                        break
                    text = sibling.get_text()
                    if text:
                        while text[0] == '\n':
                            text = text[1:]
                        if text[-1] == "\n":
                            text = text[:-1] + " "
                    plot += text
                script_data['Plot'][index] = plot
            elif("Cast" in section.get_text()):
                cast = {}
                for sibling in section.find_next_siblings():
                    if sibling.name == 'h2':
                        break
                    elif sibling.name == 'style':
                        continue
                    text = sibling.get_text()
                    text = text.split("\n")[1:-1]
                    for i in text:
                        if " as " in i:
                            cast[i.split(" as ")[0]] = i.split(" as ")[1]
                        else:
                            cast[i] = "Unknown"
                script_data['Cast'][index] = cast
        urls = soup.find_all('a', class_ = "external text")
        imdb_url = ''
        for url in urls:
            if "imdb" in str(url):
                imdb_url = url['href']
                break
        table = soup.find('table', class_ = "infobox vevent")
        trs = table.find_all('tr')
        for tr in trs:
            if("Directed by" in tr.get_text()):
                td = tr.find('td').text
                script_data['Director'][index] = td.split('\n')
            elif("Story by" in tr.get_text()):
                td = tr.find('td').text
                script_data['Writer'][index] = td.split('\n')
            elif("Production" in tr.get_text()):
                td = tr.find('td').text
                script_data['Production'][index] = td.split('\n')
            elif("Running" in tr.get_text()):
                td = tr.find('td').text.split('\n')[0]
                if "[1]" in td:
                    td = td[:-3]
                script_data['Runtime'][index] = td
            elif("Budget" in tr.get_text()):
                td = tr.find('td').text
                script_data['Budget'][index] = td[:-3]
            elif("Box office" in tr.get_text()):
                td = tr.find('td').text
                script_data['Box office'][index] = td[:-3]
        req = Request(url=imdb_url,headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urlopen(req).read()
        soup = BeautifulSoup(webpage, 'lxml')
        values = eval(soup.find('script', type="application/ld+json").text)
        script_data['Rating'][index] = values['aggregateRating']['ratingValue']
        if(values['genre']):
            script_data['Genre'][index] = values['genre']
    except Exception as e:
        print(e)

for i in range(0, len(wiki)):
    get_movie_details(i)

script_data.to_csv('script_data.csv', index=False)

In [ ]:
script_data = script_data.drop_duplicates(subset=['Title']).reset_index(drop=True)
script_data.to_csv('script_data.csv', index=False)

## **Here**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install huggingface_hub -q

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import pandas as pd
import ast
import numpy as np
def func1(x):
    try:
        return ast.literal_eval(x)
    except:
        return np.nan
script_data = pd.read_csv('https://docs.google.com/spreadsheets/d/1X21OVNellmT5hlitSp6i942ruHy4TbMBDzRQ-VUSMRg/export?format=csv&gid=1202267698')
script_data['Cast'] = script_data['Cast'].apply(lambda x: func1(x))
script_data['Director'] = script_data['Director'].apply(lambda x: func1(x))
script_data['Writer'] = script_data['Writer'].apply(lambda x: func1(x))
script_data['Production'] = script_data['Production'].apply(lambda x: func1(x))
script_data['Genre'] = script_data['Genre'].apply(lambda x: func1(x))
script_data.head()

,Title,Plot,Cast,Director,Writer,Production,Runtime,Budget,Box office,Rating,Genre
0,Ace Ventura: Pet Detective (1994),Ace Ventura is an unorthodox Miami-based priva...,"{'Jim Carrey': 'Ace Ventura', 'Sean Young': 'L...",[Tom Shadyac],[Jack Bernstein],[Morgan Creek Productions],86 minutes,$15 million,$107.2 million,6.9,[Comedy]
1,Affliction (1997),"Rolfe Whitehouse begins the film, announcing t...","{'Nick Nolte': 'Wade Whitehouse', 'James Cobur...",[Paul Schrader],NaN,"[, Largo Entertainment, Reisman Productions, K...",114 minutes,$6 mill,$6.3 million[2],6.9,"[Drama, Mystery, Thriller]"
2,Agnes Of God (1985),"In a Roman Catholic convent in Montreal, Quebe...",{'Anne Bancroft': 'Mother Miriam Ruth (religio...,[Norman Jewison],NaN,NaN,99 minutes,$7.5 million,$25.6 mill,6.6,"[Drama, Mystery]"
3,Air Force One (1997),A joint operation between American and Russian...,"{'Gary Oldman': 'Egor Korshunov, a ruthless Ra...",[Wolfgang Petersen],NaN,"[, Columbia Pictures, Beacon Pictures, Radiant...",124 minutes,$85 million,$315.2 million,NaN,NaN
4,Airplane! (1980),Ex-fighter pilot Ted Striker is a traumatized ...,"{'Robert Hays': 'Ted Striker', 'Julie Hagerty'...","[, Jim Abrahams, David Zucker, Jerry Zucker, ]",NaN,[Paramount PicturesHoward W. Koch Productions],87 minutes[2],$3.5 million,$171 million,7.7,[Comedy]


In [ ]:
!pip install datasets -q

In [ ]:
# script_sample = script_data.sample(10)

In [ ]:
import pyarrow as pa
import pyarrow.dataset as ds
from datasets import Dataset
# hg_dataset = Dataset(pa.Table.from_pandas(script_sample.reset_index(drop=True)))
script_values = []
for i in range(script_data.shape[0]):
    title = script_data['Title'][i].replace(':', '-').replace('(','- ').replace(')', '')
    f = open("/content/SFY Scripts/" + title + ".txt", "r", encoding="utf8")
    script = f.read()
    f.close()
    script_values.append(script)
# script_sample['Script'] = script_values
script_data['Script'] = script_values

In [ ]:
!pip install transformers -q
!pip install accelerate -U -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 102.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 14.2 MB/s eta 0:00:00


In [ ]:
# df = script_sample
df = script_data.copy()

In [ ]:
df.head()

,Title,Plot,Cast,Director,Writer,Production,Runtime,Budget,Box office,Rating,Genre,Script
0,Ace Ventura: Pet Detective (1994),Ace Ventura is an unorthodox Miami-based priva...,"{'Jim Carrey': 'Ace Ventura', 'Sean Young': 'L...",[Tom Shadyac],[Jack Bernstein],[Morgan Creek Productions],86 minutes,$15 million,$107.2 million,6.9,[Comedy],Ace Ventura: Pet Detective (1994)\nby Jack Ber...
1,Affliction (1997),"Rolfe Whitehouse begins the film, announcing t...","{'Nick Nolte': 'Wade Whitehouse', 'James Cobur...",[Paul Schrader],NaN,"[, Largo Entertainment, Reisman Productions, K...",114 minutes,$6 mill,$6.3 million[2],6.9,"[Drama, Mystery, Thriller]",Affliction (1997)\nby Paul Schrader.Based on a...
2,Agnes Of God (1985),"In a Roman Catholic convent in Montreal, Quebe...",{'Anne Bancroft': 'Mother Miriam Ruth (religio...,[Norman Jewison],NaN,NaN,99 minutes,$7.5 million,$25.6 mill,6.6,"[Drama, Mystery]",Agnes Of God (1985)\nby John Pielmeier.Based o...
3,Air Force One (1997),A joint operation between American and Russian...,"{'Gary Oldman': 'Egor Korshunov, a ruthless Ra...",[Wolfgang Petersen],NaN,"[, Columbia Pictures, Beacon Pictures, Radiant...",124 minutes,$85 million,$315.2 million,NaN,NaN,Air Force One (1997)\nby Andrew Marlow.\nFADE ...
4,Airplane! (1980),Ex-fighter pilot Ted Striker is a traumatized ...,"{'Robert Hays': 'Ted Striker', 'Julie Hagerty'...","[, Jim Abrahams, David Zucker, Jerry Zucker, ]",NaN,[Paramount PicturesHoward W. Koch Productions],87 minutes[2],$3.5 million,$171 million,7.7,[Comedy],"Airplane! (1980)\nby Jim Abrahams, David Zucke..."


In [ ]:
df = df.fillna("")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 531 entries, 0 to 530
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Title       531 non-null    object
 1   Plot        531 non-null    object
 2   Cast        531 non-null    object
 3   Director    531 non-null    object
 4   Writer      531 non-null    object
 5   Production  531 non-null    object
 6   Runtime     531 non-null    object
 7   Budget      531 non-null    object
 8   Box office  531 non-null    object
 9   Rating      531 non-null    object
 10  Genre       531 non-null    object
 11  Script      531 non-null    object
dtypes: object(12)
memory usage: 49.9+ KB


In [ ]:
def func2(x):
    try:
        return ", ".join(x)
    except:
        return np.nan
df['Director'] = df['Director'].apply(lambda x: func2(x))
df['Writer'] = df['Writer'].apply(lambda x: func2(x))
df['Production'] = df['Production'].apply(lambda x: func2(x))
df['Genre'] = df['Genre'].apply(lambda x: func2(x))

In [ ]:
df.head()

,Title,Plot,Cast,Director,Writer,Production,Runtime,Budget,Box office,Rating,Genre,Script
0,Ace Ventura: Pet Detective (1994),Ace Ventura is an unorthodox Miami-based priva...,"{'Jim Carrey': 'Ace Ventura', 'Sean Young': 'L...",Tom Shadyac,Jack Bernstein,Morgan Creek Productions,86 minutes,$15 million,$107.2 million,6.9,Comedy,Ace Ventura: Pet Detective (1994)\nby Jack Ber...
1,Affliction (1997),"Rolfe Whitehouse begins the film, announcing t...","{'Nick Nolte': 'Wade Whitehouse', 'James Cobur...",Paul Schrader,,", Largo Entertainment, Reisman Productions, Ki...",114 minutes,$6 mill,$6.3 million[2],6.9,"Drama, Mystery, Thriller",Affliction (1997)\nby Paul Schrader.Based on a...
2,Agnes Of God (1985),"In a Roman Catholic convent in Montreal, Quebe...",{'Anne Bancroft': 'Mother Miriam Ruth (religio...,Norman Jewison,,,99 minutes,$7.5 million,$25.6 mill,6.6,"Drama, Mystery",Agnes Of God (1985)\nby John Pielmeier.Based o...
3,Air Force One (1997),A joint operation between American and Russian...,"{'Gary Oldman': 'Egor Korshunov, a ruthless Ra...",Wolfgang Petersen,,", Columbia Pictures, Beacon Pictures, Radiant ...",124 minutes,$85 million,$315.2 million,,,Air Force One (1997)\nby Andrew Marlow.\nFADE ...
4,Airplane! (1980),Ex-fighter pilot Ted Striker is a traumatized ...,"{'Robert Hays': 'Ted Striker', 'Julie Hagerty'...",", Jim Abrahams, David Zucker, Jerry Zucker,",,Paramount PicturesHoward W. Koch Productions,87 minutes[2],$3.5 million,$171 million,7.7,Comedy,"Airplane! (1980)\nby Jim Abrahams, David Zucke..."


In [ ]:
df.drop('Cast', axis = 1, inplace = True)

In [ ]:
inputs = []
targets = []
input_ids = []
for index, row in df.iterrows():
    input_id = str(index)
    # Combine the features into a single context string
    context = "Give me a movie script where "
    for column in row.keys():
        if row[column] and column != 'Script':
            context += column + " is " + str(row[column]) + " and "
    context = context[:-5] + "."
    inputs.append(context)
    targets.append(row['Script'])
    input_ids.append(input_id)

In [ ]:
data = {'input_id':input_ids, 'input':inputs, 'target':targets}
# Convert the dictionary into a Hugging Face Dataset
dataset = Dataset.from_dict(data)

In [ ]:
dataset = dataset.train_test_split(test_size=0.3)

In [ ]:
dataset['train'][0]

{'input_id': '47',
 'input': 'Give me a movie script where Title is Basquiat (1997) and Plot is The film is a lightly fictionalized account of Basquiat\'s life. A struggling artist living in a cardboard box in Tompkins Square Park works his way up the rungs of the New York art world in the eighties, thanks in part to his association with Andy Warhol, the art dealer Bruno Bischofberger, poet and critic René Ricard, and fellow artist Albert Milo. Alongside the development of his artistic career, the film follows Basquiat\'s tumultuous relationship with Gina, a fellow aspiring artist he meets while she is working as a waitress at the diner he frequents with his friend Benny. Their romance is affected by Basquiat\'s affair with "Big Pink", a woman he picks up on the street,[4] and his habitual abuse of heroin. Eventually, Basquiat finds himself isolated by his fame, the death of Warhol, and his drug use. The film ends with an intertitle stating that Basquiat died of a heroin overdose on Au

In [ ]:
import torch
from torch.utils.data import Dataset
from transformers import GPT2Tokenizer

class CustomTextGenerationDataset(Dataset):
    def __init__(self, dataset, tokenizer, max_length=512):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        input_text = str(self.dataset[idx]['input'])
        target_text = str(self.dataset[idx]['target'])

        # Tokenize input and target texts
        input_ids = self.tokenizer.encode(input_text, add_special_tokens=True, max_length=self.max_length, truncation=True)
        target_ids = self.tokenizer.encode(target_text, add_special_tokens=True, max_length=self.max_length, truncation=True)

        # Convert tokenized sequences to PyTorch tensors
        input_ids = torch.tensor(input_ids, dtype=torch.long)
        target_ids = torch.tensor(target_ids, dtype=torch.long)

        return {"input_ids": input_ids, "labels": target_ids}

In [ ]:
class CustomTextGenerationTestDataset(Dataset):
    def __init__(self, dataset, tokenizer, max_length=512):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        input_text = str(self.dataset[idx]['input'])
        target_text = str(self.dataset[idx]['target'])

        # Tokenize input and target texts
        input_ids = self.tokenizer.encode(input_text, add_special_tokens=True, max_length=self.max_length, truncation=True)
        target_ids = self.tokenizer.encode(target_text, add_special_tokens=True, max_length=self.max_length, truncation=True)

        # Convert tokenized sequences to PyTorch tensors
        input_ids = torch.tensor(input_ids, dtype=torch.long)
        target_ids = torch.tensor(target_ids, dtype=torch.long)

        return {"input_ids": input_ids, "labels": target_ids}

In [ ]:
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling

# Load pre-trained model and tokenizer
model_name = "distilgpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

(…)e.co/distilgpt2/resolve/main/config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

(…)gpt2/resolve/main/generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

(…)ce.co/distilgpt2/resolve/main/vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

(…)ce.co/distilgpt2/resolve/main/merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

(…)o/distilgpt2/resolve/main/tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
# Create custom datasets
train_dataset = CustomTextGenerationDataset(dataset['train'], tokenizer)
test_dataset = CustomTextGenerationTestDataset(dataset['test'], tokenizer)

# Create data collator
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
tokenizer.save_pretrained('/content/my_model_50')
tokenizer.push_to_hub('newmoviescript')

CommitInfo(commit_url='https://huggingface.co/sahilnagaralu/newmoviescript/commit/3fed74a2f08512643133b98e1429d1d42cc81a31', commit_message='Upload tokenizer', commit_description='', oid='3fed74a2f08512643133b98e1429d1d42cc81a31', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
len(train_dataset)

371

In [ ]:
len(test_dataset)

160

In [ ]:
# import pickle
# pickle.dump(lm_dataset, open('/content/lm_dataset_50.pkl', 'wb'))

In [ ]:
# import pickle
# lm_dataset = pickle.load(open('/content/drive/MyDrive/SEM 5/NLP_Project/lm_dataset.pkl', 'rb'))

In [ ]:
training_args = TrainingArguments(
    output_dir="newmoviescript",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,3.609117
2,No log,3.570240
3,No log,3.558427


TrainOutput(global_step=141, training_loss=3.6878800628878547, metrics={'train_runtime': 504.5577, 'train_samples_per_second': 2.206, 'train_steps_per_second': 0.279, 'total_flos': 145411641704448.0, 'train_loss': 3.6878800628878547, 'epoch': 3.0})

In [ ]:
trainer.save_model("/content/my_model_50")

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/328M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.54k [00:00<?, ?B/s]

In [ ]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 35.11


In [ ]:
trainer.push_to_hub("newmoviescript")

'https://huggingface.co/sahilnagaralu/newmoviescript/tree/main/'

In [ ]:
# from transformers import pipeline

prompt = "Give me a movie script where title is Batman and director is Christopher Nolan and Plot is Batman tries to save many places in this world. But he is stuck in India and has no way of going out. To help him comes Shaktiman."
# generator = pipeline("text-generation", model="https://huggingface.co/yroshan/moviescript")
# generator(prompt)

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.8 MB/s eta 0:00:00


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 75.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.9 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("yroshan/moviescript")
model = AutoModelForCausalLM.from_pretrained("yroshan/moviescript")
prompt = "Give a movie script where title is Snapshot and the plot is a guy captures a photo of a rare bird species."
inputs = tokenizer(prompt, return_tensors="pt").input_ids
outputs = model.generate(inputs, max_new_tokens=300000, do_sample=True, top_k=50, top_p=0.95, pad_token_id=tokenizer.eos_token_id)
tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

'Give a movie script where title is Snapshot and the plot is a guy captures a photo of a rare bird species. (Cops are often called the "flying foxes" but "flying foxes" refer to them simply as pigeons. They do not see much, though some species resemble flying foxes, especially some the pigeons who fly in close proximity. This is because the most common bird is the "dog" named after a dog, and the person nicknamed "dog" after that dog is called "dog," due to its close-ups. However, some species of the "dog" named after a man (usually a man or a woman) are named after the dog named "dog" and the person named "dog" is named after a man, but the man named "dog" is a woman. However, some species of the "dog" named after a man (usually a man or a woman) are named after the dog named "dog" and the person named "dog" is a woman. There are several other species of the "dog" named after a man (usually a woman) and the person named "dog" is a woman. Some species of the "dog" named after a man (us